In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 8.2 MB 54.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=31e451d8d479d914fb08bbfadfea5b19344a17938ba2156e5c29eb6e3e5b62d3
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [4]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 29.1 MB/s 
     |████████████████████████████████| 216 kB 72.3 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=567ca9f41d41ea07a839da4b6e95449a3aee3ade8494f14f2d5c2bc9e8a3ba91
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [5]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 36.3 MB/s 
     |████████████████████████████████| 1.2 MB 64.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
numba 0.56.2 requires setuptools<60, but you have setuptools 65.4.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-10-11 16:54:39.999950: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 66.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [125]:
import numpy as np
import pandas as pd

Взял случайный текст Хлебникова, надеясь, что встретиться много неологизмов. Их, правда, оказалось не очень много

In [102]:
with open('/content/text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [103]:
text

'Где небо пролито из синего кувшина Из рук русалки темной площади И алошарая вершина Светла венком стеклянной проседи Ученым глазом в ночь иди Ее на небо устремленный глаз В чернила ночи ярко пролит Сорвать покровы напоказ Дворец для толп упорно волит Чтоб созерцать ряды созвездий И углублять закон возмездий Где одинокая игла На страже улицы угла Стеклянный путь покоя над покоем Был зорким стражем тишины Со стен цветным прозрачным роем Смотрели старцы вещуны В потоке золотого куполе Они смотрели мудрецы Искали правду пытали глупо ли С сынами сеть ведут отцы И шуму всего человечества Внимало спокойное жречество Но книгой черных плоскостей Разрежет город синеву И станет больше и синей Пустотный ночи круг Над глубиной прозрачных улиц В стекле тяжелом в глубине Священных лиц ряды тянулись С огнем небес наедине Разрушив жизни грубый кокон Толпа прозрачно светлых окон Под шаровыми куполами Былых видений табуны Былых времен расскажет сны В высоком и отвесном храме Здесь рода смертного отцы Вз

Использовал анализаторы pymorphy2, stanza и spacy. Тэги в разборах и в ручной разметке приводил к типу разметки pymorphy2, поскольку она более заточена на русский язык и более подробно разграничивает части речи (отдельными частями речи являются, например, инфинитив, краткое и полное причастие и деепричастие)

In [91]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [145]:
def from_pymorphy(text):
    tags = []
    for word in text.split():
        new_wd = {}
        new_wd['text'] = word
        new_wd['pos'] = morph.parse(word)[0].tag.POS
        tags.append(new_wd)
    return tags

In [146]:
razbor_pymorphy = from_pymorphy(text)
len(razbor_pymorphy)

200

In [116]:
import stanza

nlp_stanza = stanza.Pipeline(lang='ru', processors='tokenize,pos')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [155]:
def stanza_to_pymorphy(doc):
    convert = {
        'ADP':'PREP',
        'NUM': 'NUMR',
        'CONJ': 'CONJ',
        'CCONJ': 'CONJ',
        'SCONJ': 'CONJ',
        'DET': 'ADJF',
        'INTJ': 'INTJ',
        'PROPN': 'NOUN',
        'PRON': 'NPRO',
        'PART': 'PRCL',
        'NOUN': 'NOUN',
        'ADV': 'ADVB',
        'PUNCT': 'PNCT'
    }
    tags = []
    for sent in doc.sentences:
        for word in sent.words:
            new_wd = {}
            new_wd['text'] = word.text
            if word.upos == 'VERB' or word.upos == 'AUX':
                if 'VerbForm=Conv' in word.feats:
                    new_wd['pos'] = 'GRND'
                    tags.append(new_wd)
                    continue
                elif 'VerbForm=Part' in word.feats:
                    if 'Variant=Short' in word.feats:
                        new_wd['pos'] = 'PRTS'
                        tags.append(new_wd)
                        continue
                    else:
                        new_wd['pos'] = 'PRTF'
                        tags.append(new_wd)
                elif 'VerbForm=Inf' in word.feats:
                    new_wd['pos'] = 'INFN'
                    tags.append(new_wd)
                    continue
                else:
                    new_wd['pos'] = 'VERB'
                    tags.append(new_wd)
                    continue
            elif word.upos == 'ADJ':
                if 'Degree=Cmp' in word.feats:
                    new_wd['pos'] = 'COMP'
                    tags.append(new_wd)
                    continue
                elif 'Variant=Short' in word.feats:
                    new_wd['pos'] = 'ADJS'
                    tags.append(new_wd)
                    continue
                else:
                    new_wd['pos'] = 'ADJF'
                    tags.append(new_wd)
                    continue
            else:
                new_wd['pos'] = convert[word.upos]
                tags.append(new_wd)
                continue
    return tags

In [156]:
doc_stanza = nlp_stanza(text)
razbor_stanza = stanza_to_pymorphy(doc_stanza)
len(razbor_stanza)

200

In [107]:
import spacy
nlp_spacy = spacy.load("ru_core_news_sm")

In [152]:
def spacy_to_pymorphy(doc):
    convert = {
        'ADP':'PREP',
        'NUM': 'NUMR',
        'CONJ': 'CONJ',
        'CCONJ': 'CONJ',
        'SCONJ': 'CONJ',
        'DET': 'ADJF',
        'INTJ': 'INTJ',
        'PROPN': 'NOUN',
        'PRON': 'NPRO',
        'PART': 'PRCL',
        'NOUN': 'NOUN',
        'ADV': 'ADVB',
        'PUNCT': 'PNCT'
    }
    tags = []
    for word in doc:
            new_wd = {}
            new_wd['text'] = word.text
            if word.pos_ == 'VERB' or word.pos_ == 'AUX':
                if 'VerbForm=Conv' in word.morph:
                    new_wd['pos'] = 'GRND'
                    tags.append(new_wd)
                    continue
                elif 'VerbForm=Part' in word.morph:
                    if 'Variant=Short' in word.morph:
                        new_wd['pos'] = 'PRTS'
                        tags.append(new_wd)
                        continue
                    else:
                        new_wd['pos'] = 'PRTF'
                        tags.append(new_wd)
                        continue
                elif 'VerbForm=Inf' in word.morph:
                    new_wd['pos'] = 'INFN'
                    tags.append(new_wd)
                    continue
                else:
                    new_wd['pos'] = 'VERB'
                    tags.append(new_wd)
                    continue
            elif word.pos_ == 'ADJ':
                if 'Degree=Cmp' in word.morph:
                    new_wd['pos'] = 'COMP'
                    tags.append(new_wd)
                    continue
                elif 'Variant=Short' in word.morph:
                    new_wd['pos'] = 'ADJS'
                    tags.append(new_wd)
                    continue
                else:
                    new_wd['pos'] = 'ADJF'
                    tags.append(new_wd)
                    continue
            else:
                new_wd['pos'] = convert[word.pos_]
                tags.append(new_wd)
                continue
    return tags

In [153]:
doc_spacy = nlp_spacy(text)
razbor_spacy = spacy_to_pymorphy(doc_spacy)
len(razbor_spacy)

200

Считаем результаты. Лучший результат - у pymorphy2

In [158]:
true_tags = pd.read_csv('/content/tags.csv')
true_tags.head()

,text,pos
0,где,ADVB
1,небо,NOUN
2,пролито,PRTS
3,из,PREP
4,синего,ADJF


In [131]:
from sklearn.metrics import accuracy_score

In [159]:
def eval(true_tags, pred_tags):
    y_true = np.array(true_tags['pos'])
    y_pred = np.array([tag['pos'] for tag in pred_tags])
    return accuracy_score(y_true, y_pred)

In [160]:
eval(true_tags, razbor_pymorphy)

0.985

In [161]:
eval(true_tags, razbor_stanza)

0.925

In [162]:
eval(true_tags, razbor_spacy)

0.92